In [7]:
#!/usr/bin/env python
#-*- encoding: utf-8 -*-

import requests
import time
import re
from lxml import etree
import pandas as pd

def is_all_chinese(strs):
    for _char in strs:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False
    return True

def getList():
    start_url = 'https://book.douban.com/tag/青春?&type=T&start={}'
    i = 0
    infoList = []
    print(0)
    while i < 6: 
        print(i)
        url = start_url.format(i*20)
        kv = {'User-Agent': 'Mozilla/5.0'} 
        r = requests.get(url,headers=kv) 
        selector = etree.HTML(r.text)
        i += 1
        informations = selector.xpath('.//ul[@class="subject-list"]/li')
        
        for inf in informations: 
            info_bookname = inf.xpath('normalize-space(.//div[@class="info"]/h2/a/@title)')
            info_str = inf.xpath('normalize-space(.//div[@class="pub"]/text())')#[0].replaceAll("\r|\n", "")
            info_author = ''
            strlist = info_str.split('/')
            k = 0
            index = 0
            while k < len(strlist):
                if("出版" in strlist[k]):
                    index = k
                    break
                k += 1
            if(k == len(strlist)):
                info_publisher = '***'
                s=0
                index1 = 0
                while s < len(strlist):
                    if("." in strlist[s]):
                        info_price = strlist[s]
                        #index1 = s
                        index = s-1
                        break
                    s += 1
                if(s == len(strlist)):
                    info_price = strlist[len(strlist)-1]
                    index = len(strlist)-2
            else:
                info_publisher = strlist[index]#.replaceAll("\r|\n", "")
                info_price = strlist[index+2]#.replace('.','')#.replaceAll("\r|\n", "")
            j = 0
            while j < index:
                info_author += strlist[j]
                j += 1
                
            if("元" not in info_price):
                info_price = info_price+"元" 
            info_score = inf.xpath('normalize-space(.//span[@class="rating_nums"]/text())')#[0].replaceAll("\r|\n", "")
            info_commentsnum = inf.xpath('normalize-space(.//span[@class="pl"]/text())').replace('(','')
            info_commentsnum = info_commentsnum.replace(')','')
            info_commentsnum = info_commentsnum.replace('评价','')
            info_paper_edi_price = inf.xpath('normalize-space(.//span[@class="buy-info"]/a/text())')
            paper = '纸质版'
            if(paper not in info_paper_edi_price):
                info_paper_edi_price = '***'
            else:
                info_paper_edi_price = info_paper_edi_price.replace('纸质版 ','')
            if(is_all_chinese(info_bookname)):
                infoList.append([info_bookname,info_author,info_publisher,info_price,info_paper_edi_price,info_score,info_commentsnum])
        time.sleep(10)
    return infoList


def listToCsv(list):
    df = pd.DataFrame(list,columns=['书名','作者','出版社','售价','纸质版售价','豆瓣评分','评论数量'])
    file = open("1-5.csv", "w", encoding='utf-8')
    df.to_csv(file, line_terminator="\n", index=False)
    file.close()
    print(df)
    return list
 
def main():
    List = getList() 
    listToCsv(List)
 
 
if __name__=='__main__': 
	main()

0
0
1
2
3
4
5
            书名                    作者          出版社       售价    纸质版售价 豆瓣评分  \
0           撒野                   巫哲     北京联合出版公司    45.00元      ***  8.6   
1      梦里花落知多少                  郭敬明      春风文艺出版社    20.00元   7.20元起  7.1   
2        挪威的森林        [日] 村上春树  林少华      上海译文出版社    18.80元   16.00元  8.0   
3      麦田里的守望者  [美国] J. D. 塞林格  孙仲旭        译林出版社    28.00元   22.10元  8.1   
4   那些回不去的年少时光                   桐华      江苏文艺出版社    23.80元   23.80元  8.4   
..         ...                   ...          ...      ...      ...  ...   
82         他的国                   韩寒       万卷出版公司    25.00元   19.70元  7.5   
83      我的鸵鸟先生                   含胭    江苏凤凰文艺出版社    59.80元   41.20元  8.3   
84       少年巴比伦                   路内        重庆出版社    22.00元   16.00元  8.6   
85          潮骚        （日）三岛由纪夫  唐月梅      上海译文出版社    15.00元  38.70元起  8.2   
86        天气之子         [日] 新海诚  林青华     百花洲文艺出版社    42.00元   31.50元  7.2   

       评论数量  
0    30664人  
1   166899人  
2   323330人  
3    65521人  
4  